In [1]:
from google.colab import drive

drive.mount('/content/drive')

#!pip install --upgrade openai


Mounted at /content/drive


In [2]:
import pandas as pd

# Load your original tweet dataset
column_names = ['target','ids','date','flag','user','text']
df = pd.read_csv("/content/drive/MyDrive/fp/training.1600000.processed.noemoticon.csv", encoding='latin-1', names=column_names)

# Reproduce the original shuffling (random_state=42)
df_sampled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract your GPT chunk (middle third)
chunk_2 = df_sampled.iloc[30000:60000].copy()


In [3]:
df_all = df_sampled.iloc[:90000].copy()
len(df_all)

90000

In [4]:
from google.colab import files

# Save DataFrame to CSV
df_all.to_csv('df_all.csv', index=False)

# Download the file
files.download('df_all.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json

with open("gpt_batch_input.jsonl", "w") as f:
    for idx, row in chunk_2.iterrows():
        prompt = (
            f"Paraphrase the following tweet using different words while keeping the same meaning and tone. "
            f"Do not include any extra commentary.\n\n"
            f"Tweet: \"{row['text']}\"\nParaphrased:"
        )
        json.dump({
            "custom_id": str(idx),
            "messages": [{"role": "user", "content": prompt}]
        }, f)
        f.write("\n")


KeyboardInterrupt: 

In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get("OPENAI_API_KEY")  # Or use os.getenv()


In [ ]:
file_obj = openai.files.create(
    file=open("gpt_batch_input.jsonl", "rb"),
    purpose="batch"
)

input_file_id = file_obj.id
print("📁 Uploaded file ID:", input_file_id)


📁 Uploaded file ID: file-W2mAAUQNihQ38keQzdhREm


In [ ]:
import json

test_prompt = (
    "Paraphrase the following tweet using different words while keeping the same meaning and tone. "
    "Do not include any extra commentary.\n\n"
    "Tweet: \"I can't believe I spilled coffee again. What a morning.\"\nParaphrased:"
)

with open("gpt_test_batch_input.jsonl", "w") as f:
    json.dump({
        "custom_id": "test1",
        "messages": [{"role": "user", "content": test_prompt}]
    }, f)
    f.write("\n")


In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get("OPENAI_API_KEY")  # or use os.getenv()

file_obj = openai.files.create(
    file=open("gpt_test_batch_input.jsonl", "rb"),
    purpose="batch"
)

test_file_id = file_obj.id
print("📁 Test File ID:", test_file_id)


📁 Test File ID: file-A1juYBVHw6aayJ7VtEhhUm


In [ ]:
import json

output_path = "/content/drive/MyDrive/266/fp/gpt_batch/openai_batch_requests.jsonl"
test_tweets = chunk_2.head(5)  # for testing, or use .iloc[...].copy() for the real batch

prefix = "Paraphrase the following tweet using different words while keeping the same meaning and tone. Do not include hashtags, usernames, or any commentary.\n\nTweet: "
system_message = "You are a helpful assistant who rewrites social media posts using different phrasing without changing the meaning or tone."

with open(output_path, "w") as file:
    for idx, tweet in enumerate(test_tweets["text"]):
        payload = {
            "custom_id": f"tweet-{idx}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o",  # or gpt-4o-mini if that's what you want
                "messages": [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prefix + tweet}
                ],
                "max_tokens": 60
            }
        }
        file.write(json.dumps(payload) + "\n")


In [ ]:
import json

path = '/content/drive/MyDrive/fp/gpt_batch/batch_67def110afb481909d51b0d235ac86bb_output.jsonl'

data = []
with open(path, 'r') as f:
    for line in f:
        if line.strip():  # skip empty lines
            data.append(json.loads(line))

# Show first 3 entries
print(data)


[{'id': 'batch_req_67def16127388190af6eab53c630bf87', 'custom_id': 'tweet-0', 'response': {'status_code': 200, 'request_id': '69f2f8ec38eacf1937c0d50aa8b860e7', 'body': {'id': 'chatcmpl-BDwnDPLy33uOgkmB6jkQ8nGfl0VpA', 'object': 'chat.completion', 'created': 1742663963, 'model': 'gpt-4o-2024-08-06', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Enjoying some ice cream right now.', 'refusal': None, 'annotations': []}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 68, 'completion_tokens': 9, 'total_tokens': 77, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'service_tier': 'default', 'system_fingerprint': 'fp_90d33c15d4'}}, 'error': None}, {'id': 'batch_req_67def16135548190880979660179552f', 'custom_id': 'tweet-1', 'response': {'status_code': 200, 'request_id': '8f20eeef739e101d9c551

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df.head()


,id,custom_id,response,error
0,batch_req_67def16127388190af6eab53c630bf87,tweet-0,"{'status_code': 200, 'request_id': '69f2f8ec38...",None
1,batch_req_67def16135548190880979660179552f,tweet-1,"{'status_code': 200, 'request_id': '8f20eeef73...",None
2,batch_req_67def16144f88190810f867086345bec,tweet-2,"{'status_code': 200, 'request_id': 'ed0218848f...",None
3,batch_req_67def161547881908468e3226318f1b4,tweet-3,"{'status_code': 200, 'request_id': '0509ae8e47...",None
4,batch_req_67def16166088190b4fa90208ffcd8cf,tweet-4,"{'status_code': 200, 'request_id': 'c1beba08ac...",None


In [ ]:
response = df['response'].iloc[0]


In [ ]:
response['body']['choices'][0]['message']['content']


'Enjoying some ice cream right now.'

In [ ]:
def extract_paraphrased_tweet(response):
    try:
        return response['body']['choices'][0]['message']['content']
    except (TypeError, KeyError, IndexError):
        return None

df['paraphrased_tweet'] = df['response'].apply(extract_paraphrased_tweet)


In [ ]:
df[['custom_id', 'paraphrased_tweet']].head()


,custom_id,paraphrased_tweet
0,tweet-0,Enjoying some ice cream right now.
1,tweet-1,It seems the new Kylie Minogue album is droppi...
2,tweet-2,I just have brownnnn.
3,tweet-3,Thanks—I'm icing it now. Typing with my left h...
4,tweet-4,I'm so excited to see my best friend tonight. ...


In [ ]:
import pandas as pd
import json

# Load and shuffle the original tweet dataset
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
df = pd.read_csv("/content/drive/MyDrive/fp/training.1600000.processed.noemoticon.csv",
                 encoding='latin-1', names=column_names)

df_sampled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract middle third (chunk 2)
chunk_2 = df_sampled.iloc[30000:60000].copy()

# Save the GPT batch input to JSONL format
with open("gpt_batching_input_chunk2.jsonl", "w") as f:
    for idx, row in chunk_2.iterrows():
        prompt = (
            f"Paraphrase the following tweet using different words while keeping the same meaning and tone. "
            f"Do not include any extra commentary.\n\n"
            f"Tweet: \"{row['text']}\"\nParaphrased:"
        )
        json.dump({
            "custom_id": f"chunk2-tweet-{idx}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o",
                "messages": [
                    {"role": "user", "content": prompt}
                ]
            }
        }, f)
        f.write("\n")


In [ ]:
from google.colab import files
files.download("gpt_batching_input_chunk2.jsonl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.8 MB/s eta 0:00:00


In [ ]:
import tiktoken
import json

enc = tiktoken.encoding_for_model("gpt-4o")

max_tokens_per_batch = 85000  # stay below 90k
current_tokens = 0
batch_idx = 1
batch_data = []

def estimate_tokens(text):
    return len(enc.encode(text))

def save_batch(data, batch_idx):
    with open(f"gpt_batch_4omini_part_{batch_idx}.jsonl", "w") as f:
        for item in data:
            json.dump(item, f)
            f.write("\n")

for idx, row in chunk_2.iterrows():
    tweet = row['text']
    prompt = (
        f"Paraphrase the following tweet using different words while keeping the same meaning and tone. "
        f"Do not include any extra commentary.\n\n"
        f"Tweet: \"{tweet}\"\nParaphrased:"
    )
    est_tokens = estimate_tokens(prompt) + 10  # safety buffer

    item = {
        "custom_id": f"chunk2-tweet-{idx}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [{"role": "user", "content": prompt}]
        }
    }

    if current_tokens + est_tokens > max_tokens_per_batch:
        save_batch(batch_data, batch_idx)
        batch_idx += 1
        batch_data = [item]
        current_tokens = est_tokens
    else:
        batch_data.append(item)
        current_tokens += est_tokens

# save the last batch
if batch_data:
    save_batch(batch_data, batch_idx)


In [ ]:
from google.colab import files
import glob

# Automatically download all JSONL batch files
for filename in glob.glob("gpt_batch_part_*.jsonl"):
    files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil
import glob

# Move all batch files to your Drive folder
for filename in glob.glob("gpt_batch_4omini_part_*.jsonl"):
    shutil.move(filename, f"/content/drive/MyDrive/fp/gpt_batch/4omini/{filename}")


In [ ]:
import pandas as pd

# Convert JSON to DataFrame
df_output = pd.DataFrame([
    {
        "custom_id": r["custom_id"],
        "paraphrased": r["response"]["body"]["choices"][0]["message"]["content"]
    }
    for r in responses
    if r["response"]["status_code"] == 200 and r.get("response")  # safety check
])

# Preview
df_output


,custom_id,paraphrased
0,chunk2-tweet-30000,"""I'm having some ice cream"""
1,chunk2-tweet-30001,Looks like Kylie Minogue's new album is droppi...
2,chunk2-tweet-30002,"""@xEvie All I have is brownnnn"""
3,chunk2-tweet-30003,"""@MIHappenC Thanks, I'm icing it now. Typing w..."
4,chunk2-tweet-30004,"""Can't wait to hang out with my closest friend..."
...,...,...
1366,chunk2-tweet-31366,"""not watching Tsunami YouTube videos anymore, ..."
1367,chunk2-tweet-31367,"""Where's Kimm? I miss them so much."""
1368,chunk2-tweet-31368,"""@lindalrichards from the ones filled with jel..."
1369,chunk2-tweet-31369,"""@Swizec simple as that, no struggle no progre..."


In [ ]:
import tiktoken
import json

encoding = tiktoken.encoding_for_model("gpt-4o")

def estimate_tokens(prompt):
    return len(encoding.encode(prompt))

total_tokens = 0
with open("/content/drive/MyDrive/fp/gpt_batch/gpt_batch_part_2.jsonl") as f:
    for line in f:
        obj = json.loads(line)
        prompt = obj["body"]["messages"][0]["content"]
        total_tokens += estimate_tokens(prompt)

print(f"Estimated total tokens in batch 2: {total_tokens}")


Estimated total tokens in batch 2: 71243


In [ ]:
import json

input_file = "/content/drive/MyDrive/fp/gpt_batch/gpt_batch_part_2.jsonl"
output_base = "/content/drive/MyDrive/fp/gpt_batch/gpt_batch_part_2_split"

with open(input_file, "r") as f:
    lines = [line for line in f if line.strip()]

half = len(lines) // 2

with open(f"{output_base}_1.jsonl", "w") as f1:
    f1.writelines(lines[:half])

with open(f"{output_base}_2.jsonl", "w") as f2:
    f2.writelines(lines[half:])


In [ ]:
from openai import OpenAI
from google.colab import userdata

# Authenticate using Colab secrets (or manually insert your key if needed)
gpt_client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY")
)


In [ ]:
batch_input_file = gpt_client.files.create(
    file=open("/content/drive/MyDrive/fp/gpt_batch/gpt_batch_part_5.jsonl", "rb"),
    purpose="batch"
)
print("✅ Uploaded file ID:", batch_input_file.id)


✅ Uploaded file ID: file-NzsyG98DRgh7yyJYjdX6MC


In [ ]:
batch_job = gpt_client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h"  # or "4h" if needed
)
print("✅ Submitted batch. ID:", batch_job.id)


✅ Submitted batch. ID: batch_67e08f46cf948190b6f9f455e6457798


In [ ]:
batch_status = gpt_client.batches.retrieve(batch_job.id)
print("📊 Current status:", batch_status.status)


📊 Current status: failed


In [ ]:
from openai import OpenAI
import time
import os

# 🔑 Step 1: Set up your API client
from google.colab import userdata
gpt_client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# 📁 Step 2: Define file directory and batch numbers to submit
base_path = "/content/drive/MyDrive/fp/gpt_batch/4omini/"
batch_range = range(1, 23)  # Submits batches 5 through 22

def submit_and_wait(file_path):
    print(f"\n📤 Uploading: {file_path}")
    upload = gpt_client.files.create(
        file=open(file_path, "rb"),
        purpose="batch"
    )
    file_id = upload.id
    print(f"✅ Uploaded file ID: {file_id}")

    # 🚀 Submit the batch
    batch = gpt_client.batches.create(
        input_file_id=file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h"
    )
    batch_id = batch.id
    print(f"🚀 Submitted batch ID: {batch_id}")

    # ⏳ Wait for completion
    while True:
        status = gpt_client.batches.retrieve(batch_id).status
        print(f"⏳ Batch {batch_id} status: {status}")
        if status in ["completed", "failed", "cancelled", "expired"]:
            break
        time.sleep(20)

    return batch_id, status

# 🔁 Step 3: Submit each batch file, stop on failure
for i in batch_range:
    file_path = os.path.join(base_path, f"gpt_batch_4omini_part_{i}.jsonl")

    if not os.path.exists(file_path):
        print(f"⚠️ File not found: {file_path}")
        continue

    try:
        batch_id, status = submit_and_wait(file_path)
        print(f"🎯 Batch {batch_id} finished with status: {status.upper()}")

        # 🛑 Stop script if batch didn't complete
        if status != "completed":
            print(f"❌ Batch failed or cancelled. Stopping here.")
            break

    except Exception as e:
        print(f"💥 Error during batch submission for {file_path}: {e}")
        break



📤 Uploading: /content/drive/MyDrive/fp/gpt_batch/4omini/gpt_batch_4omini_part_1.jsonl
✅ Uploaded file ID: file-3XgLzKFFrNyFby5CYFYWNb
🚀 Submitted batch ID: batch_67e0aca0726c8190a51828ad91534680
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: validating
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳ Batch batch_67e0aca0726c8190a51828ad91534680 status: in_progress
⏳